## Extract erroneous scraped symbols
We use this code to extract symbols that encountered errors during the scraping process. Subsequently, we will re-scrape these symbols

In [10]:
import re
import os
import pandas as pd

def extract_symbols(log_folder_path):
    warning_symbols = set()
    info_symbols = set()

    # Regex patterns to identify WARNING and INFO logs
    warning_pattern = re.compile(r"WARNING - Not enough reports (\w+):")
    info_pattern = re.compile(r"INFO - Scraping Successfully (\w+):")

    # Loop through each log file in the folder
    for filename in os.listdir(log_folder_path):
        if filename.endswith(".log"):
            with open(os.path.join(log_folder_path, filename), 'r') as file:
                for line in file:
                    # Match WARNING logs
                    warning_match = warning_pattern.search(line)
                    if warning_match:
                        symbol = warning_match.group(1)
                        warning_symbols.add(symbol)

                    # Match INFO logs
                    info_match = info_pattern.search(line)
                    if info_match:
                        symbol = info_match.group(1)
                        info_symbols.add(symbol)

    # Exclude symbols that appear in INFO logs from WARNING symbols
    final_warning_symbols = warning_symbols - info_symbols

    # Create a DataFrame to display the results
    warning_df = pd.DataFrame(final_warning_symbols, columns=["symbol"])
    print("Symbols with warnings (excluding those in INFO logs):")
    print(warning_df)

    return warning_df



In [11]:
# Example usage
log_folder_path = "./logging"  # Replace with your log folder path
warnings_df = extract_symbols(log_folder_path)

Symbols with warnings (excluding those in INFO logs):
  symbol
0    SAB
1    NVL
2    CTG


In [ ]:
# warnings_df.to_csv("error_symbols.csv")

In [17]:
import json
for symbol in list(warnings_df["symbol"]):
    checkpoint_path = f"./checkpoints/checkpoint_{symbol}.json"
    checkpoint = {"current_page": 1, "last_row_index": 0}

    # Save the updated checkpoints back to the file
    with open(checkpoint_path, 'w') as f:
        json.dump(checkpoint, f)

In [22]:
pd.read_csv("E:\PUBLIC_DATA_PROJECT\Optimizing_scraping\crawl_table\CTG\warning.csv")

,Unnamed: 0,STT,Tên báo cáo,Đơn vị,Trích yếu,Thời gian gửi,Mã doanh nghiệp,Tên công ty,Tiêu đề,Saving_path
0,0,1,Báo cáo tài chính Mẹ - Quý,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính riêng lẻ Quý III/2024 và giả...,30/10/2024 00:00:00,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính Mẹ - Quý,./crawl_table/CTG/Báo cáo tài chính riêng lẻ Q...
1,1,2,Báo cáo tài chính Hợp nhất - Quý,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính hợp nhất Quý III/2024 và giả...,30/10/2024 00:00:00,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính Hợp nhất - Quý,./crawl_table/CTG/Báo cáo tài chính hợp nhất Q...
2,2,3,Báo cáo tài chính Mẹ - Bán niên,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính 6 tháng đầu năm 2024 riêng l...,14/08/2024 00:00:00,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính Mẹ - Bán niên,./crawl_table/CTG/Báo cáo tài chính 6 tháng đầ...
3,3,4,Báo cáo tài chính Hợp nhất - Bán niên,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính 6 tháng đầu năm 2024 hợp nhấ...,14/08/2024 00:00:00,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính Hợp nhất - Bán niên,./crawl_table/CTG/Báo cáo tài chính 6 tháng đầ...
4,4,5,Báo cáo tài chính quý 2/ năm 2024,Ngân hàng TMCP Công thương Việt Nam,Ngân hàng Công Thương kính gửi báo cáo tài chí...,30/07/2024 21:59:35,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính quý 2/ năm 2024,./crawl_table/CTG/Ngân hàng Công Thương kính g...
...,...,...,...,...,...,...,...,...,...,...
95,95,6,Báo cáo tài chính hợp nhất quý 2/ năm 2024,Ngân hàng TMCP Công thương Việt Nam,Ngân hàng TMCP Công thương Việt Nam kính gửi B...,30/07/2024 21:54:33,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính hợp nhất quý 2/ năm 2024,./crawl_table/CTG/Ngân hàng TMCP Công thương V...
96,96,7,Báo cáo tài chính quý 1/ năm 2024,Ngân hàng TMCP Công thương Việt Nam,NHTMCP Công Thương Kính gửi Báo cáo tài chính ...,26/04/2024 17:04:16,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính quý 1/ năm 2024,./crawl_table/CTG/NHTMCP Công Thương Kính gửi ...
97,97,8,Báo cáo tài chính hợp nhất quý 1/ năm 2024,Ngân hàng TMCP Công thương Việt Nam,Ngân hàng TMCP Công thương VN kính gửi BCTC hợ...,26/04/2024 10:49:08,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính hợp nhất quý 1/ năm 2024,./crawl_table/CTG/Ngân hàng TMCP Công thương V...
98,98,9,Báo cáo tài chính hợp nhất năm 2023,Ngân hàng TMCP Công thương Việt Nam,Ngân hàng TMCP Công thương Việt Nam kính gửi B...,29/03/2024 21:13:03,100111948,Ngân hàng TMCP Công thương Việt Nam,Báo cáo tài chính hợp nhất năm 2023,./crawl_table/CTG/Ngân hàng TMCP Công thương V...
